In [ ]:
# lib reloading
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [ ]:
# python standard packages
import logging 
import os
import sys
from pathlib import Path

# third-party packages
import keyring
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# in project packages
sys.path.append(str(Path(os.getcwd()).parent))  # add parent folder to paths
from utilities.chrome_driver import ChromeDriver

# logging configuration
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# project constants
ELEMENT_TIMEOUT = 2  # [s]

# needed to be setup earlier
# keyring.set_password("credential_name", "user_name", "password")
UNITY_CREDENTIALS = ("unity_id_cred", "email_address")

In [ ]:
chrome_driver = ChromeDriver().chrome_driver
selenium_waiter = WebDriverWait(chrome_driver, timeout=ELEMENT_TIMEOUT)
chrome_driver.get("https://assetstore.unity.com/")
logging.info("Asset Store open")

In [ ]:
accept_all_cookies_btn = chrome_driver.find_element(By.ID, "onetrust-accept-btn-handler")
selenium_waiter.until(EC.element_to_be_clickable(accept_all_cookies_btn))
accept_all_cookies_btn.click()
logging.info("Cookies Accepted")

In [ ]:
chrome_driver.get("https://assetstore.unity.com/auth/login?redirect_to=%2F")
login_page = chrome_driver.current_url

In [ ]:
# write email
email_field = chrome_driver.find_element(By.ID, "conversations_create_session_form_email")
selenium_waiter.until(EC.element_to_be_clickable(email_field))
email_field.send_keys(UNITY_CREDENTIALS[1])

In [ ]:
# write password
password_field = chrome_driver.find_element(By.ID, "conversations_create_session_form_password")
selenium_waiter.until(EC.element_to_be_clickable(password_field))
password_field.send_keys(keyring.get_password(*UNITY_CREDENTIALS))

In [ ]:
# click submit
submit_btn = chrome_driver.find_element(By.XPATH, "//input[@type='submit']")
selenium_waiter.until(EC.element_to_be_clickable(submit_btn))
submit_btn.click()

In [ ]:
# check if proper page and credentials are ok
page = chrome_driver.page_source
page_soup = BeautifulSoup(page, "html.parser")
unity_asset_store_desc = "Unity Asset Store - The Best Assets"
if not unity_asset_store_desc in page_soup.text:
    if chrome_driver.current_url == login_page:
        raise Exception("Bad Credentials")
    else:
        raise Exception("Wrong Page obtained after login, maybe 2FA enabled?")

In [ ]:
# go to proper adress url with query url

In [ ]:
chrome_driver.quit()